In [1]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [2]:
df = pd.read_csv('wiki_movie_plots_deduped.csv')
df.shape

(34886, 8)

# Preprocessing

In [3]:
df = df[df['Genre'] != 'unknown']
df.shape

(28803, 8)

In [4]:
df.head(60)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...
11,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...
12,1906,Kathleen Mavourneen,American,Edwin S. Porter,NaN,short film,https://en.wikipedia.org/wiki/Kathleen_Mavourn...,Irish villager Kathleen is a tenant of Captain...
13,1907,Daniel Boone,American,Wallace McCutcheon and Ediwin S. Porter,"William Craven, Florence Lawrence",biographical,https://en.wikipedia.org/wiki/Daniel_Boone_(19...,Boone's daughter befriends an Indian maiden as...
14,1907,How Brown Saw the Baseball Game,American,Unknown,Unknown,comedy,https://en.wikipedia.org/wiki/How_Brown_Saw_th...,Before heading out to a baseball game at a nea...
15,1907,Laughing Gas,American,Edwin Stanton Porter,"Bertha Regustus, Edward Boulden",comedy,https://en.wikipedia.org/wiki/Laughing_Gas_(fi...,The plot is that of a black woman going to the...
16,1908,The Adventures of Dollie,American,D. W. Griffith,"Arthur V. Johnson, Linda Arvidson",drama,https://en.wikipedia.org/wiki/The_Adventures_o...,On a beautiful summer day a father and mother ...
17,1908,The Black Viper,American,D. W. Griffith,D. W. Griffith,drama,https://en.wikipedia.org/wiki/The_Black_Viper,A thug accosts a girl as she leaves her workpl...


In [5]:
df['Genre'].nunique()

2264

In [6]:
# 1.1. dictionary-based conversion readacting Wikipedia's Film genre page:

conversion_dict = {
    "action": ["disaster", "martial arts", "spy", "superhero", "wuxia","action","masala","espionage","arts"],
    "adventure": ["pirate", "swashbuckler", "samurai"],
    "animation": ["cgi", "cutout", "live-action animated film", "stop motion", "animated", "computer-animated", "anime"],
    "comedy": ["buddy", "mockumentary", "parody", "slapstick"],
    "drama": ["docudrama", "melodrama", "biodrama", "bio-drama"],
    "historical": ["history", "historic", "alternate history", "period", "period piece", "biopic", "bio-pic", "biographical"],
    "horror": ["ghost", "monster", "vampire", "werewolf", "slash", "splatter", "zombie", "j-horror","supernatural"],
    "science fiction": ["dystopian", "dystopia", "post-apocalyptic", "steampunk", "tech noir", "utopian", "science-fiction", "scifi", "sci-fi", "space", "tokusatsu","fiction"],
    "thriller": ["mystery", "detective", "crime","suspense"],
    "musical": ["operetta"],
    "romance": ["love","romantic"],
    "western": ["cowboy"],
    "documentary": ["pseudo-documentary"],
    "fantasy":[],
    "sport":["sports","races","dance","biker"],
    "war":['ii','i'],
    "erotic":['ero','adult','erotic','sexploitation'],
    "social":['socio','costume']
}

In [7]:
import re

#preprocessing function helper for genres reduction
def genres_preprocessing(genres_dict, genre):
    stop = ['film','short']
    for w in stop:
        if w in genre:
            genre = genre.replace(w,'').strip()
        
    
    splitted = re.split("[,/]", genre)
    if len(splitted) != 1:
        genre = splitted[0]
        
    splitted = re.split("[-—–]", genre)
    if any(item in genres_dict.keys() for item in splitted):
    #if splitted[0] in genres_dict.keys():
        genre = splitted[0]
    
    splitted = re.split(" ", genre.rstrip())
    if len(splitted) != 1:
        genre = splitted[-1]
        
    for key in genres_dict.keys():
        if genre.rstrip() in genres_dict[key]:
            genre = key
        
    return genre.rstrip()

In [8]:
df['Genre2'] = df['Genre']
df['Genre2'] = df['Genre2'].apply(lambda x: genres_preprocessing(conversion_dict, x))

In [9]:
counts = df['Genre2'].value_counts()
to_remove = counts[counts < 50].index

df = df[~df.Genre2.isin(to_remove)]

drop_id = df[df.Plot.apply(lambda x: len(x) < 25)].index
df.drop(drop_id, inplace=True)

df = df[df['Genre2'] != '']

In [591]:
#g = df.groupby("Genre2")
#new_df = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

In [140]:
vc = df['Genre2'].value_counts()
genres_to_fetch = vc[vc < 1000].index.tolist()
genres_to_fetch

['western',
 'animation',
 'science fiction',
 'adventure',
 'musical',
 'war',
 'noir',
 'family',
 'fantasy',
 'historical',
 'biography',
 'social',
 'documentary',
 'serial',
 'sport']

In [220]:
import requests
from bs4 import BeautifulSoup

def get_links(src, stw="/", genre=""):
    links = []
    soup = BeautifulSoup(src)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href and href.startswith(stw) and genre in href.lower():
            links.append(href)
            
    return links

def request_links(base, in_links):
    out_links = []
    if type(in_links) == list:
        response = [ requests.get(base+link).text for link in set(in_links) ]
        for res in response:
            out_links.extend(get_links(res))

    else:
        response = requests.get(base+in_links).text
        out_links.extend(get_links(response))
        
    return out_links

base = "https://en.wikipedia.org"
genres_link = "https://en.wikipedia.org/wiki/Template:Films_by_genre_sidebar"

In [221]:
main_res = requests.get(genres_link).text
main_links = []
for genre in genres_to_fetch:
    main_links.extend(get_links(main_res, "/wiki/List", genre))
main_links
#request_links(base, genres_link)

['/wiki/Lists_of_Western_films',
 '/wiki/Lists_of_adventure_films',
 '/wiki/List_of_musical_films_by_year',
 '/wiki/List_of_war_films_and_TV_specials',
 '/wiki/List_of_anti-war_films',
 '/wiki/List_of_film_noir_titles',
 '/wiki/List_of_neo-noir_films',
 '/wiki/Lists_of_fantasy_films',
 '/wiki/List_of_documentary_films',
 '/wiki/List_of_sports_films']

In [237]:
l = main_links[-1]
final_link = request_links(base, l)
final_link

['/wiki/Film_genre',
 '/wiki/Lists_of_action_films',
 '/wiki/Lists_of_adventure_films',
 '/wiki/Lists_of_animated_feature_films',
 '/wiki/List_of_stop_motion_films',
 '/wiki/Lists_of_avant-garde_films',
 '/wiki/List_of_biographical_films',
 '/wiki/List_of_children%27s_films',
 '/wiki/Lists_of_comedy_films',
 '/wiki/Lists_of_crime_films',
 '/wiki/List_of_disaster_films',
 '/wiki/List_of_documentary_films',
 '/wiki/List_of_drama_films',
 '/wiki/List_of_economics_films',
 '/wiki/Lists_of_erotic_films',
 '/wiki/Lists_of_fantasy_films',
 '/wiki/Lists_of_horror_films',
 '/wiki/List_of_natural_horror_films',
 '/wiki/List_of_independent_short_films',
 '/wiki/List_of_LGBT-related_films',
 '/wiki/List_of_mockumentaries',
 '/wiki/List_of_monster_movies',
 '/wiki/List_of_musical_films_by_year',
 '/wiki/List_of_mystery_films',
 '/wiki/List_of_film_noir_titles',
 '/wiki/List_of_neo-noir_films',
 '/wiki/List_of_religious_films',
 '/wiki/List_of_romantic_comedy_films',
 '/wiki/List_of_satirical_films'

In [272]:
link = "https://en.wikipedia.org/wiki/Million_Dollar_Baby"
html = requests.get(link).text
soup = BeautifulSoup(html)
plot_span = soup.find("span", {"id":"Plot"})
obj = plot_header.find_parent()
paragraphs = []
while True:
    obj = obj.next_sibling
    if obj.name not in ['p','h2']:
        continue
    elif obj.name == 'p':
        paragraphs.append(obj.text)
    elif obj.name == 'h2':
        break

In [273]:
paragraphs

['Margaret "Maggie" Fitzgerald (Hilary Swank), a waitress from the Ozarks, shows up at the Hit Pit, a rundown Los Angeles gym owned and operated by Frankie Dunn (Clint Eastwood). Dunn is a cantankerous Irish-American trainer, revealed to be estranged from his daughter. Maggie asks Frankie to train her, but he refuses as he doesn\'t train women and she\'s too old to begin a boxing career. Eddie "Scrap-Iron" Dupris (Morgan Freeman), Frankie\'s friend and employee — and the film\'s narrator — encourages and helps Maggie.\n',
 'Frankie\'s prize prospect, "Big Willie" Little, signs with successful manager Mickey Mack after becoming impatient with Frankie rejecting offers for a championship bout. Frankie then reluctantly agrees to train Maggie. \n',
 "Maggie fights her way up in the women's amateur boxing division with Frankie's coaching. Since she has earned a reputation for quick KOs, Frankie must resort to bribery to get other managers to put their trainee fighters up against her.\n",
 'S

# Classification

### SVM

In [10]:
from sklearn.model_selection import train_test_split

X = df['Plot'].values
y = df['Genre2'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [11]:
len(X_train),len(y_train),len(X_test),len(y_test)

(21029, 21029, 7010, 7010)

In [12]:
import nltk

nlp = spacy.load('en_core_web_sm')

doc_counter = 0
def reset_counter():
  global doc_counter
  doc_counter = 0

def increase_counter():
  global doc_counter
  doc_counter += 1
  if doc_counter % 100 == 0:
    print(doc_counter)

def spacy_nlp_tokenizer(text):
    increase_counter()
        
    # we use spacy for main nlp tasks
    doc = nlp(text)
    # lemmatized tokens, skipping stopwords
    lemmas = ['LEMMA_'+token.lemma_ for token in doc if not token.is_stop]
    # entity_types
    entity_types = ['NER_'+token.ent_type_ for token in doc if token.ent_type_]

    # in case an entity linker is available, we can use it do put actual entities as
    # features, e.g. Queen Elizabeth, Elizabeth II, Her Majesty -> KB2912
    # see https://spacy.io/usage/training#entity-linker
    # entities = ['ENT_'+token.ent_kb_id_ for token in doc if token.ent_kb_id_]

    # we use a simple nltk function to create ngrams
    lemma_bigrams = ['BI_'+p1+'_'+p2 for p1,p2 in nltk.ngrams(lemmas,2)]
    lemma_trigrams = ['TRI_'+p1+'_'+p2+'_'+p3 for p1,p2,p3 in nltk.ngrams(lemmas,3)]

    all_tokens = list()
    all_tokens.extend(lemmas)
    all_tokens.extend(lemma_bigrams)
    all_tokens.extend(lemma_trigrams)
    all_tokens.extend(entity_types)
    return all_tokens

In [13]:
vect = CountVectorizer(analyzer=spacy_nlp_tokenizer, min_df=5)

X_train_tok = vect.fit_transform(X_train)

X_test_tok = vect.transform(X_test)

100
200
300
400
500
600
700
800
900


KeyboardInterrupt: 

In [ ]:
X_train_tok.shape, X_test_tok.shape

In [ ]:
import pickle

with open('x_train_tok.pkl','wb') as outfile:
    pickle.dump(X_train_tok,outfile)
with open('x_test_tok.pkl','wb') as outfile:
    pickle.dump(X_test_tok,outfile)

In [12]:
import pickle

infile = open('x_train_tok.pkl','rb')
X_train_tok = pickle.load(infile)
infile.close()

infile = open('x_test_tok.pkl','rb')
X_test_tok = pickle.load(infile)
infile.close()

In [13]:
X_train_tok.shape, X_test_tok.shape

((21029, 6806178), (7010, 6806178))

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'sel__k': [1000, 2000, 5000, 10000], 'learner__C': [0.01, 0.1, 1, 10]},
 ]

opt_pipeline = Pipeline([
    ('sel', SelectKBest(chi2)),  # feature selection
    ('tfidf', TfidfTransformer()),  # weighting
    ('learner', LinearSVC())  # learning algorithm
])


n_jobs = 2

opt_search = GridSearchCV(opt_pipeline, param_grid, cv=5, n_jobs = n_jobs, verbose=3).fit(X_train_tok,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [15]:
opt_predictions = opt_search.best_estimator_.predict(X_test_tok)

correct = 0
for prediction,true_label in zip(opt_predictions, y_test):
    if prediction==true_label:
        correct += 1
print(correct/len(opt_predictions))

0.2905848787446505


In [16]:
from sklearn.metrics import confusion_matrix, classification_report
print('Classification report:')
print(classification_report(y_test, opt_predictions))
print('Confusion matrix:')
cm = confusion_matrix(y_test, opt_predictions)
print(cm)

Classification report:
                 precision    recall  f1-score   support

         action       0.00      0.00      0.00       516
      adventure       0.00      0.00      0.00       166
      animation       0.00      0.00      0.00       205
           arts       0.00      0.00      0.00        18
      biography       0.00      0.00      0.00        55
         comedy       0.24      0.02      0.04      1636
    documentary       0.00      0.00      0.00        23
          drama       0.29      0.98      0.45      2041
         family       0.00      0.00      0.00        81
        fantasy       0.00      0.00      0.00        78
        fiction       0.00      0.00      0.00       117
     historical       0.00      0.00      0.00        63
         horror       0.00      0.00      0.00       342
        musical       0.00      0.00      0.00       139
           noir       0.00      0.00      0.00        86
        romance       0.00      0.00      0.00       335
science

C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacopo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
